In [46]:
import torch
from skimage import io
import onnxruntime as ort
import onnx
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1"

In [57]:
from onnx_opcounter import calculate_params


In [58]:
params = calculate_params(model)


In [59]:
params

9157346

In [47]:
model = onnx.load('densenet_model.onnx')


In [48]:
from onnx2torch import convert


In [49]:
torch_model_1 = convert(model)


NotImplementedError: Converter is not implemented (OperationDescription(domain='', operation_type='GlobalMaxPool', version=1))

In [50]:
try:
    onnx.checker.check_model('densenet_model_batch.onnx')
except onnx.checker.ValidationError as e:
    print(f"The model is invalid: {e}")
else:
    print("The model is valid!")


The model is valid!


In [4]:
model = onnx.load('densenet_model.onnx')
model.graph.input[0].type.tensor_type.shape.dim[0].dim_param = 'batch_size'
model.graph.input[0].type.tensor_type.shape.dim[0].ClearField('dim_value')
onnx.save(model, 'densenet_model_batch.onnx')


In [20]:
total_params = sum(p for p in model.parameters())
learnable_params = sum(p for p in model.parameters() if p.requires_grad)


AttributeError: parameters

In [42]:
model.__srt

AttributeError: __name__

In [7]:
ort_session = ort.InferenceSession("densenet_model.onnx", None, providers=["CUDAExecutionProvider"])

In [18]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
from utils import Multilabel_classifier
import torch
import vision_transformer as vits
from file_dataset import ImageFileList, AutoBalancedFileList, default_loader, pandas_reader, pandas_reader_no_labels, pandas_reader_binary_labels
from tqdm import tqdm
from yaml_tfms import tfms_from_config
import numpy as np
from label_dict import protein_to_num_full as protein_to_num
from sklearn.metrics import f1_score
import yaml
import pandas as pd
from pathlib import Path
import torch
import importlib
import file_dataset
from label_dict import protein_to_num_full
importlib.reload(file_dataset)
ImageFileList = file_dataset.ImageFileList
pandas_reader_binary_labels = file_dataset.pandas_reader_binary_labels
from tqdm import tqdm
import argparse
import torch
import importlib
import utils
importlib.reload(utils)
from utils import Multilabel_classifier, simple_clf, expanding_clf, prototyping_clf, residual_clf, residual_add_clf
import protein_dataset
importlib.reload(protein_dataset)
from protein_dataset import ProteinDataset 

batch_size_per_gpu = 512
IDs = (torch.load('/scr/mdoron/Dino4Cells/models/FAIR_whole_image_all_channels/features.pth')[3])

dataset = ProteinDataset(
        IDs,
        img_size=1536,
        is_trainset=False,
        return_label=False,
        in_channels=4,
        transform=None,
        crop_size=1024,
        random_crop=False,
    )

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
IDs = []
features = []
predictions = []
for ind, (img, ID) in tqdm(enumerate(dataset), total=len(dataset)):
    ort_outs = ort_session.run(None, {'image': to_numpy(img[np.newaxis,:,:,:])})
    IDs.append(ID)
    predictions.append(ort_outs[0])
    features.append(ort_outs[1])

sigmoid_predictions = (torch.Tensor(predictions)).round().squeeze(1)[:, mapping]

submissions = []
for i in (sigmoid_predictions):
    if len(i) == 0:
        submissions.append('')
    else:
        submissions.append(' '.join([str(v) for v in np.where(i)[0]]))
        
# submission = pd.DataFrame(zip(dataset.img_ids, submissions), columns=['Id', 'Predicted'])
# submission = submission.sort_values(by='Id')
# submission.to_csv(f'bestfitting_submission.csv', index=False)
# os.system('kaggle competitions submit -c human-protein-atlas-image-classification -f bestfitting_submission.csv -m bestfitting -q')


ValueError: Invalid file path or buffer object type: <class 'list'>

In [16]:
all_features = torch.zeros(len(dataset), 1024).float()
all_proteins = torch.zeros(len(dataset), 28).int()


In [25]:
ort_session.get_inputs()


In [19]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0,1,2,3,4,5,6,7"
from utils import Multilabel_classifier
import torch
import vision_transformer as vits
from file_dataset import ImageFileList, AutoBalancedFileList, default_loader, pandas_reader, pandas_reader_no_labels, pandas_reader_binary_labels
from tqdm import tqdm
from yaml_tfms import tfms_from_config
import numpy as np
from label_dict import protein_to_num_full as protein_to_num
from sklearn.metrics import f1_score
import yaml
import pandas as pd
from pathlib import Path
import torch
import importlib
import file_dataset
from label_dict import protein_to_num_full
importlib.reload(file_dataset)
ImageFileList = file_dataset.ImageFileList
pandas_reader_binary_labels = file_dataset.pandas_reader_binary_labels
from tqdm import tqdm
import argparse
import torch
import importlib
import utils
importlib.reload(utils)
from utils import Multilabel_classifier, simple_clf, expanding_clf, prototyping_clf, residual_clf, residual_add_clf
import protein_dataset
importlib.reload(protein_dataset)
from protein_dataset import ProteinDataset 

batch_size_per_gpu = 1

dataset = ProteinDataset(
        '/scr/mdoron/Dino4Cells/data/kaggle_whole_image_test_IDs.csv',
        img_size=1536,
        is_trainset=False,
        return_label=False,
        in_channels=4,
        transform=None,
        crop_size=1024,
        random_crop=False,
        target_labels=sorted(list(protein_to_num_full.keys()))
)

kaggle_data_loader = torch.utils.data.DataLoader(
    dataset,
#     sampler=train_sampler,
    batch_size=batch_size_per_gpu,
    num_workers=10,
    pin_memory=True,
    drop_last=False,
)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
IDs = []
features = []
cell_lines = []
proteins = []
bad_IDs = []
outputs = []
all_features = torch.zeros(len(dataset), 1024).float()
all_proteins = torch.zeros(len(dataset), 28).float()
i = 0
for img, ID in tqdm(kaggle_data_loader):
# for img, protein, cell, ID in tqdm(kaggle_data_loader):
    img1 = img
    try:
        ort_outs = ort_session.run(None, {'image': to_numpy(img)})
        IDs.extend(ID)
#         cell_lines.extend(cell)
#         all_proteins[i,:] = protein[0]
#         all_features[i,:] = torch.Tensor(ort_outs[1][0])
        outputs.append(ort_outs[0])
#         i += 1
#         del ort_outs, img, protein, cell, ID
    except:
        bad_IDs.append(ID)

# torch.save((all_features, all_proteins, cell_lines, IDs), '/scr/mdoron/Dino4Cells/models/bestfitting_whole/features.pth')        



/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/test/images_1536
/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/train/external_v18_1536


  1%|██                                                                                                                                                                 | 150/11702 [00:10<13:12, 14.57it/s]


KeyboardInterrupt: 

In [26]:
torch.save((all_proteins, all_features, cell_lines, IDs), '/scr/mdoron/Dino4Cells/models/bestfitting_whole/features.pth')        


In [98]:
predictions = (np.stack(outputs)[:,0,:] > 0.5).astype(int)

In [99]:
submissions = []
for i in predictions:
    if len(i) == 0:
        submissions.append("")
    else:
        submissions.append(" ".join([str(v) for v in np.where(i)[0]]))

submission = pd.DataFrame(zip(IDs, submissions), columns=["Id", "Predicted"])
submission = submission.sort_values(by="Id")

In [100]:
submission.to_csv('/scr/mdoron/Dino4Cells/results/bestfitting_whole/submission.csv', index=False)

In [ ]:
ls '/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/' -lctr

In [64]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="1"
from utils import Multilabel_classifier
import torch
import vision_transformer as vits
from file_dataset import ImageFileList, AutoBalancedFileList, default_loader, pandas_reader, pandas_reader_no_labels, pandas_reader_binary_labels
from tqdm import tqdm
from yaml_tfms import tfms_from_config
import numpy as np
from label_dict import protein_to_num_full as protein_to_num
from sklearn.metrics import f1_score
import yaml
import pandas as pd
from pathlib import Path
import torch
import importlib
import file_dataset
from label_dict import protein_to_num_full
importlib.reload(file_dataset)
ImageFileList = file_dataset.ImageFileList
pandas_reader_binary_labels = file_dataset.pandas_reader_binary_labels
from tqdm import tqdm
import argparse
import torch
import importlib
import utils
importlib.reload(utils)
from utils import Multilabel_classifier, simple_clf, expanding_clf, prototyping_clf, residual_clf, residual_add_clf
import protein_dataset
importlib.reload(protein_dataset)
from protein_dataset import ProteinDataset 

batch_size_per_gpu = 32

dataset = ProteinDataset(
        '/mnt/data0/mdoron/HPA_data/whole_images_large_test.csv',
        img_size=1536,
        is_trainset=True,
        return_label=True,
        in_channels=4,
        transform=None,
        crop_size=1024,
        random_crop=False,
        target_labels=sorted(list(protein_to_num_full.keys()))
)

kaggle_data_loader = torch.utils.data.DataLoader(
    dataset,
#     sampler=train_sampler,
    batch_size=batch_size_per_gpu,
    num_workers=10,
    pin_memory=True,
    drop_last=False,
)

def to_numpy(tensor):
    return tensor.detach().cpu().numpy() if tensor.requires_grad else tensor.cpu().numpy()

# compute ONNX Runtime output prediction
IDs = []
features = []
cell_lines = []
proteins = []
bad_IDs = []
outputs = []
for img, ID in tqdm(kaggle_data_loader):
# for img, protein, cell, ID in tqdm(kaggle_data_loader):
    try:
        ort_outs = ort_session.run(None, {'image': to_numpy(img)})
        outputs.append(ort_outs)
#         IDs.extend(ID)
#         cell_lines.extend(cell)
#         proteins.extend(protein)
#         features.append(ort_outs[1])
    except:
        bad_IDs.append(ID)

# torch.save((features, proteins, cell_lines, IDs), '/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/test_features.pth')        

/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/test/images_1536
/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/train/external_v18_1536


  0%|▍                                                                                                                                                                      | 1/366 [00:03<22:25,  3.69s/it]


KeyboardInterrupt: 

In [53]:

dataset = ProteinDataset(
        '/scr/mdoron/Dino4Cells/data/kaggle_whole_image_test_IDs.csv',
        img_size=1536,
        is_trainset=True,
        return_label=False,
        in_channels=4,
        transform=None,
        crop_size=1024,
        random_crop=False,
        target_labels=[]
)

/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/train/images_1536
/mnt/data0/mdoron/HPA_data/HPA-competition-solutions/bestfitting/data/protein/train/external_v18_1536


In [67]:
img.shape

torch.Size([32, 4, 1024, 1024])

In [ ]:
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   # see issue #152
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from utils import Multilabel_classifier
import torch
import vision_transformer as vits
from file_dataset import ImageFileList, AutoBalancedFileList, default_loader, pandas_reader, pandas_reader_no_labels, pandas_reader_binary_labels
from tqdm import tqdm
from yaml_tfms import tfms_from_config
import numpy as np
from label_dict import protein_to_num_full as protein_to_num
from sklearn.metrics import f1_score
import yaml
import pandas as pd
from pathlib import Path
import torch
import importlib
import file_dataset
from label_dict import protein_to_num_full
importlib.reload(file_dataset)
ImageFileList = file_dataset.ImageFileList
pandas_reader_binary_labels = file_dataset.pandas_reader_binary_labels
from tqdm import tqdm
import argparse
import torch
import importlib
import utils
importlib.reload(utils)
from utils import Multilabel_classifier, simple_clf, expanding_clf, prototyping_clf, residual_clf, residual_add_clf

config = 'configs/config_supervised_ViT.yaml'
config = yaml.safe_load(open(config, 'r'))
class transformation(object):
    def __init__(self, config, isTrain):
        self.isTrain = isTrain
        self.config = config
        self.global_transfo1, self.global_transfo2, self.local_transfo, self.testing_transfo = tfms_from_config(self.config)
    def __call__(self, image):
        train_crop = self.global_transfo1(image)
        test_crop = self.testing_transfo(image)
        if self.isTrain:
            return train_crop
        else:
            return test_crop

train_transform = transformation(config, isTrain=True)
test_transform = transformation(config, isTrain=False)

batch_size_per_gpu = 512
dataset = ImageFileList('/mnt/data0/mdoron/HPA_data/kaggle_test.csv', 
                               transform=test_transform, 
                               loader = default_loader, 
                               flist_reader = pandas_reader_no_labels, 
                               root=config['model']['root'],
                               with_labels=False,
                               training=False,
                               target_labels=None,
                               balance=False)

data_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size=batch_size_per_gpu,
    num_workers=10,
    pin_memory=True,
    drop_last=False,
)

IDs = []
cell_lines = []
features = []
targets = []
with torch.no_grad():
    for images, ID in tqdm(data_loader):
        features.append(model(images.cuda()).detach().cpu())
        IDs.extend(ID)